In [161]:
import os
import pickle

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [162]:
df_2002301 = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
)

In [163]:
df_2002301.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [164]:
df_2002301.shape

(3066766, 19)

**Q1. Answer** = 19 (How many columns are there?)

## Q2. Computing duration
What's the standard deviation of the trips duration in January?



In [165]:
df_2002301["tpep_dropoff_datetime"].isna().sum()

0

In [166]:
df_2002301["tpep_pickup_datetime"].isna().sum()

0

In [167]:
df = df_2002301.copy()

In [168]:
df["duration"] = df_2002301["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]

In [169]:
df["duration"] = df["duration"].apply(
    lambda time_delta: time_delta.total_seconds() / 60
)

In [170]:
df["duration"].describe(percentiles=[0.90, 0.92, 0.95, 0.98]).apply(
    lambda x: format(x, "f")
)

count    3066766.000000
mean          15.668995
std           42.594351
min          -29.200000
50%           11.516667
90%           27.941667
92%           30.550000
95%           36.466667
98%           48.733333
max        10029.183333
Name: duration, dtype: object

**Q2. Answer** = 42. (What's the standard deviation of the trips duration in January?)

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [171]:
df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]
df["duration"].describe().apply(lambda x: format(x, "f"))

count    3009173.000000
mean          14.204864
std            9.939386
min            1.000000
25%            7.216667
50%           11.550000
75%           18.183333
max           60.000000
Name: duration, dtype: object

In [172]:
fraction = len(df) / len(df_2002301)
fraction

0.9812202822125979

**Q3. Answer** = 98% (What fraction of the records left after you dropped the outliers?)

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [173]:
df_2002301["PULocationID"].isna().sum()

0

In [174]:
df_2002301["DOLocationID"].isna().sum()

0

In [175]:
categorical = ["PULocationID", "DOLocationID"]
df[categorical] = df[categorical].astype(str)

In [176]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [177]:
train_dicts = df[categorical].to_dict(orient="records")
train_dicts[:10]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'}]

In [178]:
len(train_dicts)

3009173

In [179]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [180]:
X_train.shape

(3009173, 515)

In [181]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

**Q4. Answer** = 515 (What's the dimensionality of this matrix (number of columns)?)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?



In [182]:
y_train = df["duration"].values
y_train

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)

rmse

7.649262029221019

**Q5. Answer** = 7.64 (What's the RMSE on train?)

## Extra: Save model

In [183]:
directory_path = "models"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created.")
else:
    print(f"Directory '{directory_path}' already exists.")

with open("models/lin_reg_homework.bin", "wb") as f_out:
    pickle.dump((dv, model), f_out)

Directory 'models' already exists.


In [184]:
del model
del X_train
del y_train
del y_pred
del df_2002301
del df
del dv

In [185]:
# pd.reset_option("all", silent=True)
# gc.collect()

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [186]:
with open("models/lin_reg_homework.bin", "rb") as file:
    dv, model = pickle.load(file)

In [187]:
df_2002302 = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
)

In [188]:
df_2002302.shape

(2913955, 19)

In [189]:
df_2002302.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [190]:
df_2002302.isna().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          76817
trip_distance                0
RatecodeID               76817
store_and_fwd_flag       76817
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     76817
Airport_fee              76817
dtype: int64

In [191]:
df = df_2002302.copy()

df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]

df["duration"] = df["duration"].apply(
    lambda time_delta: time_delta.total_seconds() / 60
)

df["duration"].describe(percentiles=[0.90, 0.92, 0.95, 0.98]).apply(
    lambda x: format(x, "f")
)

count    2913955.000000
mean          16.015910
std           42.842102
min          -43.616667
50%           11.800000
90%           28.483333
92%           31.116667
95%           37.216667
98%           49.900000
max         7053.616667
Name: duration, dtype: object

In [192]:
df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]
df["duration"].describe().apply(lambda x: format(x, "f"))

count    2855951.000000
mean          14.468110
std           10.064227
min            1.000000
25%            7.366667
50%           11.816667
75%           18.600000
max           60.000000
Name: duration, dtype: object

In [193]:
categorical = ["PULocationID", "DOLocationID"]

val_dicts = df[categorical].astype(str).to_dict(orient="records")
val_dicts[:5]

[{'PULocationID': '142', 'DOLocationID': '163'},
 {'PULocationID': '132', 'DOLocationID': '26'},
 {'PULocationID': '161', 'DOLocationID': '145'},
 {'PULocationID': '148', 'DOLocationID': '236'},
 {'PULocationID': '137', 'DOLocationID': '244'}]

In [194]:
len(val_dicts)

2855951

In [195]:
# dv = DictVectorizer()
X_val = dv.transform(val_dicts)
X_val

<2855951x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5711894 stored elements in Compressed Sparse Row format>

In [196]:
y_val = df["duration"].values
y_val, y_val.shape

(array([ 1.68333333, 32.08333333, 13.3       , ..., 14.        ,
         7.        ,  9.8       ]),
 (2855951,))

In [197]:
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

7.8118169669491095

**Q6. Answer** = 7.81 (What's the RMSE on validation?)